# Reasoning Approaches Experimentation Notebook

This notebook allows you to experiment with different reasoning approaches across various LLM providers and models.

## Features
- Support for multiple providers: OpenRouter, Anthropic, Cohere
- 10 different reasoning approaches
- HuggingFace dataset integration
- CSV export and HuggingFace Hub upload
- Comprehensive experiment tracking

## Setup Instructions
1. Install required packages
2. Set up API keys
3. Load your dataset
4. Configure experiments
5. Run experiments
6. Analyze results


## 1. Setup and Installation


In [ ]:
# Install required packages
!pip install -q openai anthropic cohere datasets huggingface-hub pandas numpy tqdm requests python-dotenv


In [ ]:
# Import all necessary modules
import os
import pandas as pd
import numpy as np
from datetime import datetime
import json
import warnings
warnings.filterwarnings('ignore')

# Import our custom modules
from providers import ProviderManager
from reasoning_approaches_corrected import ReasoningApproachManager  # Use corrected version
from dataset_loader import DatasetLoader
from experiment_runner import ExperimentRunner

print("✅ All modules imported successfully!")
print("📚 Using corrected reasoning approaches with proper citations and algorithms")


## 2. API Keys Setup


In [ ]:
# Set up API keys
# You can either set them as environment variables or input them directly

# Option 1: Set environment variables (recommended for security)
# os.environ['OPENROUTER_API_KEY'] = 'your_openrouter_key_here'
# os.environ['ANTHROPIC_API_KEY'] = 'your_anthropic_key_here'
# os.environ['COHERE_API_KEY'] = 'your_cohere_key_here'
# os.environ['HUGGINGFACE_TOKEN'] = 'your_hf_token_here'

# Option 2: Input directly (less secure, but convenient for testing)
OPENROUTER_API_KEY = ""  # Your OpenRouter API key
ANTHROPIC_API_KEY = ""    # Your Anthropic API key
COHERE_API_KEY = ""       # Your Cohere API key
HUGGINGFACE_TOKEN = ""    # Your HuggingFace token

print("🔑 API keys configured!")


## 3. Initialize Components


In [ ]:
# Initialize all components
provider_manager = ProviderManager()
reasoning_manager = ReasoningApproachManager()
dataset_loader = DatasetLoader(hf_token=HUGGINGFACE_TOKEN)
experiment_runner = ExperimentRunner(provider_manager, reasoning_manager, dataset_loader)

# Add providers
if OPENROUTER_API_KEY:
    provider_manager.add_provider("openrouter", OPENROUTER_API_KEY)
    print("✅ OpenRouter provider added")

if ANTHROPIC_API_KEY:
    provider_manager.add_provider("anthropic", ANTHROPIC_API_KEY)
    print("✅ Anthropic provider added")

if COHERE_API_KEY:
    provider_manager.add_provider("cohere", COHERE_API_KEY)
    print("✅ Cohere provider added")

print(f"\n🧠 Available reasoning approaches: {len(reasoning_manager.get_available_approaches())}")
print(f"🏢 Available providers: {list(provider_manager.providers.keys())}")

# Show citations for research transparency
print("\n📚 Research Citations:")
citations = reasoning_manager.get_citations()
for approach, citation in citations.items():
    print(f"\n{approach}:")
    print(f"  {citation}")


## 4. Load Dataset


In [ ]:
# Load dataset - choose one option:

# Option 1: Load from HuggingFace
dataset_name = "gsm8k"  # Change to your preferred dataset
split = "test"
num_samples = 5  # Limit for testing, set to None for full dataset

print(f"📥 Loading {dataset_name} dataset...")
df = dataset_loader.load_huggingface_dataset(
    dataset_name=dataset_name,
    split=split,
    num_samples=num_samples
)

# Preprocess for our format
df = dataset_loader.preprocess_dataset(df, input_field="question", expected_field="answer")

# Option 2: Use sample dataset (uncomment to use)
# df = dataset_loader.create_sample_dataset()

print(f"✅ Dataset loaded: {len(df)} samples")
print(f"📝 Sample input: {df['input'].iloc[0][:100]}...")


## 5. Configure and Run Experiments


In [ ]:
# Configure experiments
selected_approaches = [
    "None",
    "Chain-of-Thought (CoT)",
    "Least-to-Most Prompting (LtM)",
    "Reasoning-as-Planning (RAP)"
]

selected_models = [
    "gpt-4o-mini",  # OpenRouter
    "claude-3-5-sonnet-20241022",  # Anthropic
    "command-r-plus"  # Cohere
]

max_samples = 3  # Limit for testing
output_dir = "experiment_results"

print(f"🎯 Running {len(selected_approaches)} approaches × {len(selected_models)} models × {min(max_samples, len(df))} samples")
print(f"📊 Total experiments: {len(selected_approaches) * len(selected_models) * min(max_samples, len(df))}")

# Run experiments
results_df = experiment_runner.run_batch_experiments(
    dataset=df,
    reasoning_approaches=selected_approaches,
    models=selected_models,
    max_samples=max_samples,
    output_dir=output_dir
)

print("✅ Experiments completed!")


## 6. Analyze Results


In [ ]:
# Analyze results
summary = experiment_runner.get_experiment_summary()
print("📈 Experiment Summary:")
for key, value in summary.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.2f}")
    else:
        print(f"  {key}: {value}")

# Compare approaches
print("\n🧠 Approach Comparison:")
approach_comparison = experiment_runner.compare_approaches()
print(approach_comparison)

# View sample results
print("\n📋 Sample Results:")
for i, (_, row) in enumerate(results_df.head(2).iterrows()):
    print(f"\nExperiment {i+1}:")
    print(f"Input: {row['input'][:80]}...")
    print(f"Approach: {row['reasoning_approach']}")
    print(f"Model: {row['model_name']}")
    print(f"Time: {row['execution_time_s']:.2f}s")
    print(f"Output: {row['model_output'][:100]}...")


## 7. Export Results


In [ ]:
# Export results
from google.colab import files

# Save and download CSV
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"reasoning_experiments_{timestamp}.csv"
results_df.to_csv(filename, index=False)
files.download(filename)

print(f"📥 Downloaded: {filename}")

# Optional: Push to HuggingFace Hub
push_to_hf = False  # Set to True to enable
if push_to_hf and HUGGINGFACE_TOKEN:
    repo_name = "your-username/reasoning-experiments"  # Replace with your repo
    success = experiment_runner.push_to_huggingface(
        df=results_df,
        repo_name=repo_name,
        hf_token=HUGGINGFACE_TOKEN
    )
    print(f"🚀 HuggingFace upload: {'✅ Success' if success else '❌ Failed'}")
else:
    print("⏭️  HuggingFace upload skipped")
